# Ingest Additional Rasters on Earth Engine

* Purpose of script: This notebook will ingest some of the missing rasters to Earth Engine 
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170803

## Preparation

1. gcloud authorization (`gcloud init`)
1. earthengine authorization (`earthengine authorize`)
1. aws authorization (`aws configure`)


Create working environment and copy relevant files

In [2]:
!mkdir /volumes/data/Y2017M08D02_RH_Ingest_Additional_Rasters_EE_V01